In [ ]:
!ls /dev/ttyUSB*

In [ ]:
# building the robot
import requests
from rosmaster_create_bot import CreateBot, sleep, EXTENSION_BOARD_ADDRESS, GAMEPAD_CONTROLLER_ID
    # check with:   ls /dev/ttyUSB*
    # if cant find try to reset the device, reinstall and reconnect it
extension_board_address = "/dev/ttyUSB0" or EXTENSION_BOARD_ADDRESS 
bot = CreateBot(extension_board_address)

move_arm = bot.arm
move_motor = bot.motor


In [ ]:
#functions

def turn_around(turning_time=1.5):
    move_motor.move_turn0(is_move=True)
    sleep(turning_time)  #timing for turning around
    move_motor.stop()

def move_forward():
    move_motor.move_forward(is_move=True)
    

def stop():
    move_motor.stop()

def lift_arm():
    move_arm.arm_bent_up(70)
def pull_down():
    move_arm.arm_bent_down(70)

def turn_off_the_lamp():
    url = "https://arthurcam.com/api/arduinoIOT/2"
    requests.get(url)


In [ ]:
#function running for demo

#preset
pull_down()

turn_around()
sleep()
move_forward()
sleep()
stop()
sleep()
lift_arm()
sleep()
pull_down()
sleep()
turn_around()
sleep()
#turn of the lamp

#resture
lift_arm()

In [ ]:
# check the API action on https://www.arthurcam.com
turn_off_the_lamp()

In [ ]:
#building the model
from nemo_speech import NemoSpeech
nemo_model = NemoSpeech()
#load the recorder
from recorder import VoiceRecorder
record = VoiceRecorder()
#load phrase to function
from phrase_to_function import PhraseToFunction

# api calling
import requests

#building the robot
from rosmaster_create_bot import CreateBot, sleep, EXTENSION_BOARD_ADDRESS, GAMEPAD_CONTROLLER_ID
    # check with:   ls /dev/ttyUSB*
    # if cant find try to reset the device, reinstall and reconnect it
extension_board_address = "/dev/ttyUSB0" or EXTENSION_BOARD_ADDRESS 
bot = CreateBot(extension_board_address)

move_arm = bot.arm
move_motor = bot.motor

In [ ]:
# appling Rosmater commands for the demo
# agent waiting loop for command with exiting
from agent import Agent
from recorder import VoiceRecorder
from phrase_to_function import PhraseToFunction
import time

#speach command
def say(text: str):
    nemo_model.text_to_speech(text, play=True)

# Microphone, Input callback: returns transcribed text
recorder = VoiceRecorder(mic_id=25)

def asr_input(): #resouces callback
    recorder.record()
    audio = recorder.get_audio()
    if len(audio) == 0:
        return ""
    return nemo_model.transcribe_from_audio(audio).text.strip().lower()


#functions
def turn_around(turning_time=1):
    move_motor.move_turn1(is_move=True)
    sleep(turning_time)  #timing for turning around
    move_motor.stop()

def move_forward():
    say("Yes moving forward")
    move_motor.move_forward(is_move=True)
    move_motor.move_backward(is_move=True)
    move_motor.set_speed(15)
    move_motor.move_forward(is_move=True)
    
    
def stop():
    move_motor.stop()
    move_motor.set_speed(50)

def lift_arm():
    say("Arm up")
    move_arm.arm_bent_up(70)


def pull_down():
    say("Arm down")
    move_arm.arm_bent_down(70)

def turn_off_the_lamp():
    url = "https://arthurcam.com/api/arduinoIOT/2"
    requests.get(url)


EXIT_COMMANDS = {"exit", "quit", "terminate"}

rosmaster_commands = {
    "say hi": lambda: say("Hi Rosmaster"),
    "turn around": turn_around,
    "move forward": move_forward,
    "stop": stop,
    "lift arm": lift_arm,
    "pull down": pull_down,
    "arm down": pull_down,
    "turn off the lamp": turn_off_the_lamp,
    "make beep": lambda: bot.beep()
}


# Logic callback for the agent
def agent_logic(commands, input_callback):
    recognizer = PhraseToFunction(commands)
    say("Rosmaster is waiting for the command")
    while True:
        try:
            command = input_callback()
            if not command:
                bot.beep()
                continue
            print(f"🗣 Command: '{command}'")

            if command in EXIT_COMMANDS:
                say("Exiting. Goodbye.")
                bot.beep()
                break

            recognized, result = recognizer.execute(command)
            if not recognized:
                bot.beep()
            if result:
                pass
                
        except Exception as e:
            bot.beep()
            print(f"❌ Error: {e}")
            say("Error occurred. Try again.")
            bot.beep()
            time.sleep(1)


if __name__ == "__main__":
    stop()
    pull_down()#preset command
    main_agent = Agent("MainAgent", agent_logic, asr_input, rosmaster_commands)
    main_agent.run()
    lift_arm()#reset command
    bot.terminate()


In [ ]:
stop()                               

In [ ]:
import sounddevice as sd
for i, dev in enumerate(sd.query_devices()):
    if dev['max_input_channels'] > 0:
        print(f"[{i}] {dev['name']}")

In [ ]:
import sounddevice as sd

with sd.InputStream(device=0, channels=1, samplerate=44100) as stream:
    print("Recording...")
    audio = stream.read(44100)[0]